# LSA Example
## 1. โหลด example data จากไฟล์
## 2. Prepare data
1. Remove stopword
2. Stemmer
3. TF-IDF Vectorizer

## 3. Create model ทำ SVD เพื่อให้ได้ matrix
- matrix V.T (topic-term matrix)
- matrix U (document-topic matrix)
- matrix sigma (topic strength matrix)

## 4. การนำไปใช้
- [Unsupervised Machine Learning] ใช้ matrix V.T (lsa_model.components_.T) บน new data เพื่อให้ทราบ hidden topic ของ new data(matrix U) เพื่อจัดกลุ่มตาม latent topic
- [Dimensionality Reduction for Supervised Machine Learning] ปรับ n_components ของ TruncatedSVD(n_components=5) ตามสมควรเพื่อลด dimension ของ data(เดิมเป็น document - terms) เป็น document - latent topic เพื่อใช้เป็น features ในกระบวนการ classification ต่อไป


## 1. โหลด example data จากไฟล์
#### ไฟล์ข้อมูลนี้ถูกแปลง/เตรียมมาก่อนแล้วระดับหนึ่ง
##### สามารถดูไฟล์เต็มๆได้บน <a href="https://www.kaggle.com/bittlingmayer/amazonreviews/downloads/amazonreviews.zip" target="_blank">Kaggle ที่นี่</a>

In [1]:
import nltk
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

raw_data = pd.read_csv('dataset-lsa.csv',nrows =20000)
#split data into train test
test_raw = raw_data.iloc[0:500,:]
train_raw = raw_data.iloc[500:20000,:]

## 2. Prepare data
1. Remove stopword
2. Stemmer
3. TF-IDF Vectorizer

In [2]:
# my custom preprocessor
# Use stemmer
# Filter stopword
def preprocessor(content):
    content = content.lower()
    content = re.sub(r'[^\w]', ' ', content)
    _stemmer = nltk.stem.porter.PorterStemmer()
    stopword = nltk.corpus.stopwords.words('english')
    token = nltk.tokenize.word_tokenize(content)
    new_content = ""
    for x in token:
        if x not in stopword:
            new_content += _stemmer.stem(x)+' '
    return new_content[:-1]

In [3]:
# remove stopword
# concat as a list of string
def transform(data,vectorizer = None):
    if not vectorizer:
        vectorizer = TfidfVectorizer(preprocessor=preprocessor, tokenizer=nltk.tokenize.word_tokenize)
        processedData = vectorizer.fit_transform(data)
    else:
        processedData = vectorizer.transform(data)
    return processedData, vectorizer
train_data, vectorizer = transform(train_raw.content)

## 3. Create model ทำ SVD เพื่อให้ได้ matrix V.T U Sigma หลังจากนั้น show first 15 domiant words of each topic
 1. matrix V.T (topic-term matrix)
 2. matrix U (document-topic matrix)
 3. matrix sigma (topic strength matrix)

In [4]:
#run SVD (using truncated to reduce the computational power required as it can limit the number of topics)
lsa_model = TruncatedSVD(n_components=6)
lsa_model.fit(train_data)

TruncatedSVD(algorithm='randomized', n_components=6, n_iter=5,
       random_state=None, tol=0.0)

- หากต้องการ matrix U หลังจาก fit สามารถใช้ .fit_transform แทน .fit ได้ โดยจะ return matrix U สำหรับ data
- หลังจากทำการ SVD แล้วสามารถเข้าถึง V.T U sigma ได้ดังนี้
 - V.T คือ  lsa_model.components_
 - sigma คือ lsa_model.singular_values_ ในรูปแบบของลิสต์ หากต้องการใช้ต้องสร้างเป็น diagonal matrix ก่อน
 - U หากไม่ได้ใช้ fit_transform มาตั้งแต่แรก สามารถใช้ lsa_model.transform(train_data) ใหม่ได้ เพื่อให้ return matrix U ออกมา
   - วิธีการเดียวกันสามารถนำ new data มา transfrom ได้เพื่อให้ทราบถึง latent topic จาก matrix U
   - สามารถใช้ matrix V.T เพื่อสร้าง matrix U สำหรับ new data ได้โดยใช้ docment_term_matrix dot product กับ V.T ได้ แทนการใช้ lsa_model.transform(test_data)

In [5]:
# Show important terms of each latent topic
print('important terms of each latent topic')
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:15]
    print("Topic "+str(i)+": ",end="")
    for t in sorted_terms:
        print(t[0],end=" ")
    print("")

important terms of each latent topic
Topic 0: book movi read one great good like time love get would stori realli first work 
Topic 1: book read author page write stori written novel reader interest charact inform chapter finish seri 
Topic 2: movi book watch film read stori seen act charact plot bore see funni scene actor 
Topic 3: album cd song movi music listen book best love read band favorit rock ever stori 
Topic 4: great love cd dvd price easi enjoy excel movi recommend christma book fun old son 
Topic 5: cd book movi dvd buy money wast product qualiti amazon would disappoint order sound player 


In [6]:
#Show 5 documents of each latent topic
def print_samples(y,data):
    for level in set(y):
        print('Topic :',level)
        try:
            sample_data = data.iloc[y == level].sample(n=5).content.values
        except:
            sample_data = data.iloc[y == level].content.values
        for sample in sample_data:
            print('\t-',sample)
        print('-----------------------------\n\n')

In [7]:
topic_matrix = lsa_model.transform(train_data)
y = np.argmax(topic_matrix,axis=1)
print('5 training sample documents from each latent topic')
print_samples(y,train_raw)

5 training sample documents from each latent topic
Topic : 0
	- Arrived in poor condition: I purchased the St. Joseph home sale kit for a friend who has been struggling to sell her house. It wasn't an expensive gift but it was a gift nonetheless. However  the kit was shipped in an ordinary padded envelope and its little cardboard box didn't survive the rigors of the postal service. It was dented and smashed. In addition  the cardboard packaging looked like it had been exposed to moisture  it was slightly puffed in areas and looked old and used. Overall the kit that I received was not in gift-giving condition although the contents were unharmed. It was very disappointing.
	- Damaged DVD: This DVD was damaged and I was not able to view it. It was returned and picked up on 1.4.10 and I have not yet received a credit for it. Please let me know how to proceed and receive my credit so I can purchse another DVD. I was very excited about viewing it and would still like the opty to do so. Thank

## 4. การนำไปใช้
- [Unsupervised Machine Learning] ใช้ matrix V.T (lsa_model.components_.T) บน new data เพื่อให้ทราบ hidden topic ของ new data(matrix U) เพื่อจัดกลุ่มตาม latent topic
- [Dimensionality Reduction for Supervised Machine Learning] ปรับ n_components ของ TruncatedSVD(n_components=5) ตามสมควรเพื่อลด dimension ของ data(เดิมเป็น document - terms) เป็น document - latent topic(matrix U) เพื่อใช้เป็น features ในกระบวนการ classification ต่อไป

In [8]:
# [Unsupervised Machine Learning] ใช้ matrix V.T (lsa_model.components_.T) บน new data เพื่อให้ทราบ hidden topic ของ new data(matrix U) เพื่อจัดกลุ่มตาม latent topic
test_data, vectorizer= transform(test_raw.content,vectorizer = vectorizer)
#สามารถใช้ np.dot(test_data.toarray(),lsa_model.components_.T ) แทน lsa_model.transform(test_data) ได้
topic_matrix = lsa_model.transform(test_data)
y = np.argmax(topic_matrix,axis=1)
print('Upto 5 testing sample documents from each latent topic')
print_samples(y,test_raw)

Upto 5 testing sample documents from each latent topic
Topic : 0
	- Good matt nude lipstick: It's got a nice warm scent of vanilla-like smell. Texture is creamy. Meadow Sweet is a nice nude color. Can't apply too much or else it'll look a bit dry.
	- fireplace tool set: Good product. Got it in a timely manner. Does exactly what it is supposed to do. Not too expensive and just as good as the rest of the tools that are over priced. The tools work great and have been used a lot this winter.
	- Persuasive book with an unusual approach: The book details many or most of the important dynamics of the war as it took place within a single provice. In the process Bergerud reveals much about what is important to know about the war as a whole.For me personally this is possibly the most important Vietnam book I've read. This is because it provided me a framework for understanding the war  from the beginning of American involvement to the end  that I still use.For many readers  this could be an impo

สำหรับ [Dimensionality Reduction for Supervised Machine Learning] ปรับ n_components ของ TruncatedSVD(n_components=5) ตามสมควร<br>เพื่อลด dimension ของ data(เดิมเป็น document - terms) เป็น document - latent topic เพื่อใช้เป็น features ในกระบวนการ classification ต่อไป<br>
โดยบรรทัดต่อไปนี้จะทำการทำ<br>
1. ใช้ SVD เพื่อลดจำนวน dimension ลงเหลือ7000 dimension 
2. นำ matrix U ที่ได้ไปใช้เป็น features ของการทำ KNN Classifier

In [9]:
#Reduce dimension to 70000 dimensions
lsa_model2 = TruncatedSVD(n_components=7000)
lsa_model2.fit(train_data)
features_lsa = lsa_model2.transform(train_data)
y_label = train_raw.label.tolist()
print('Original dimension:', train_data.shape)
print('New dimension:',features_lsa.shape)

Original dimension: (19500, 35076)
New dimension: (19500, 7000)


In [10]:
#create model from tf-idf as a baseline
clf_tfidf = KNeighborsClassifier()
clf_tfidf.fit(X=train_data,y=y_label)

#create model from lsa_feature
clf_lsa = KNeighborsClassifier()
clf_lsa.fit(X=features_lsa,y=y_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [11]:
#Actual y
y_test = test_raw.label.tolist()

#Transform data to reduce dimension to 70000 dimension(same as feature_lsa)
test_data_lsa = lsa_model2.transform(test_data)

In [12]:
#Perform the model prediction
y_pred_tfidf = clf_tfidf.predict(test_data)
y_pred_lsa = clf_lsa.predict(test_data_lsa)

#Compare accuracy of both model
print('Accuracy of KNN with TF-IDF: ',accuracy_score(y_test, y_pred_tfidf))
print('Accuracy of KNN with LSA: ',accuracy_score(y_test, y_pred_lsa))

Accuracy of KNN with TF-IDF:  0.71
Accuracy of KNN with LSA:  0.71
